In [22]:
# Third party imports
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [23]:
response = requests.get(
	url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",
)
print(response.status_code)

200


In [24]:
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find(id='firstHeading')
print(title.string)

List of postal codes of Canada: M


In [25]:
table_contents = []
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [27]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace(
    {
        'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
        'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
        'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
        'MississaugaCanada Post Gateway Processing Centre':'Mississauga'
    }
)

In [37]:
df.loc[df['Borough'].str.contains('Toronto', regex=False)]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches
20,M5E,Downtown Toronto,Berczy Park
24,M5G,Downtown Toronto,Central Bay Street
25,M6G,Downtown Toronto,Christie
30,M5H,Downtown Toronto,"Richmond, Adelaide, King"
31,M6H,West Toronto,"Dufferin, Dovercourt Village"
35,M4J,East York/East Toronto,The Danforth East
